In [ ]:
import numpy as np
import os
import yaml
import torch
from scipy import stats as st
from scipy.stats import t as t_dist, norm, cauchy, laplace
from funcs import Scenario2
from neural_model import QuantileNetwork

In [ ]:
def load_config(path):
    return yaml.load(open(path, 'r'), Loader=yaml.SafeLoader)

config = load_config("/Users/cxxxx_/Desktop/DeepQuantile_covariate_shift/config.yaml")


In [ ]:
N_trials = 10
N_test = 10000
sample_sizes = [512,2048]
quantiles = np.array([0.05, 0.25, 0.5, 0.75, 0.95])
n_tau = 5
functions = [Scenario2()]
models = [lambda: QuantileNetwork(quantiles=quantiles, config=config)]

    # Track the performance results
l2_results = np.full((N_trials, len(functions), len(models), len(sample_sizes), len(quantiles)), np.nan)
l1_results = np.full((N_trials, len(functions), len(models), len(sample_sizes), len(quantiles)), np.nan)

In [ ]:
alpha=3
beta=4
alpha_pr=2.5
beta_pr=1.5
n_tr=1000
n_te=1000
X_tr=np.random.uniform(0,1,size=(n_tr,3))
X_tr[:,0]=np.random.beta(alpha_pr,beta_pr,n_tr)
X_te=np.random.uniform(0,1,size=(n_te,3))
X_te[:,0]=np.random.beta(alpha,beta,n_te)

# Unweighted Estimator

In [ ]:
for trial in range(N_trials):
        print(f'Trial {trial+1}')
        for scenario, func in enumerate(functions):
            print(f'\tScenario {scenario+1}')
            
       
            X_test = np.random.uniform(0,1,size=(N_test,3))
            X_test[:,0] = np.random.beta(alpha,beta,N_test)
            y_test = func.sample(X_test)

            # Get the ground truth quantiles
            y_quantiles = np.array([func.quantile(X_test, q) for q in quantiles]).T
            

            for nidx, N_train in enumerate(sample_sizes):
                print(f'\t\tN={N_train}')
                
                X_train = np.random.uniform(0,1,size=(N_train,3))
                X_train[:,0] = np.random.beta(alpha_pr,beta_pr,N_train)
                y_train = func.sample(X_train)

                for midx, model in enumerate([m() for m in models]):
                    print(f'\t\t\t{model.label}')

    

                    model.fit(X_train, y_train)
                    preds = model.predict(X_test)

             # Evaluate the model on the ground truth quantiles
                    l2_results[trial, scenario, midx, nidx] = ((y_quantiles - preds)**2).mean(axis=0)

                    l1_results[trial, scenario, midx, nidx] = (np.abs(y_quantiles - preds)).mean(axis=0)
                    
            print('\t', l2_results[trial, scenario])
            print('\t', l1_results[trial, scenario])

# Reweighted Estimator

In [ ]:
alpha=3
beta=4
alpha_pr=2.5
beta_pr=1.5
n_tr=1000
n_te=1000

for trial in range(N_trials):
        print(f'Trial {trial+1}')
        for scenario, func in enumerate(functions):
            print(f'\tScenario {scenario+1}')
           
            # Sample test set covariates and response
            X_test = np.random.uniform(0,1,size=(N_test,3))
            X_test[:,0] = np.random.beta(alpha,beta,N_test)
            y_test = func.sample(X_test)

            # Get the ground truth quantiles
            y_quantiles = np.array([func.quantile(X_test, q) for q in quantiles]).T

            

            for nidx, N_train in enumerate(sample_sizes):
                print(f'\t\tN={N_train}')
                # Sample training covariates and response
                
                X_train = np.random.uniform(0,1,size=(N_train,3))
                X_train[:,0] = np.random.beta(alpha_pr,beta_pr,N_train)
                y_train = func.sample(X_train)
                
                true_weight = st.beta(alpha,beta).pdf(X_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(X_train[:,0]) 
              
                X_tr=np.random.uniform(0,1,size=(n_tr,3))
                X_tr[:,0]=np.random.beta(alpha_pr,beta_pr,n_tr)
                X_te=np.random.uniform(0,1,size=(n_te,3))
                X_te[:,0]=np.random.beta(alpha,beta,n_te) 

                for midx, model in enumerate([m() for m in models]):
                    print(f'\t\t\t{model.label}')

    

                    model.weighted_fit(X_train, y_train, X_tr, X_te, n_tr, true_weight, 0, 3)
                    preds = model.predict(X_test)
                    print(preds.shape)

                    # Evaluate the model on the ground truth quantiles
                    l2_results[trial, scenario, midx, nidx] = ((y_quantiles - preds)**2).mean(axis=0)

                    l1_results[trial, scenario, midx, nidx] = (np.abs(y_quantiles - preds)).mean(axis=0)
                    
            print('\t', l2_results[trial, scenario])
            print('\t', l1_results[trial, scenario])

# Pre-trained Estimator

In [ ]:
for trial in range(N_trials):
        print(f'Trial {trial+1}')
        for scenario, func in enumerate(functions):
            print(f'\tScenario {scenario+1}')
           
            # Sample test set covariates and response
            X_test = np.random.uniform(0,1,size=(N_test,3))
            X_test[:,0] = np.random.beta(alpha,beta,N_test)
            y_test = func.sample(X_test)

            # Get the ground truth quantiles
            y_quantiles = np.array([func.quantile(X_test, q) for q in quantiles]).T

            risk = np.full((N_test, n_tau), np.nan) 
            

            for nidx, N_train in enumerate(sample_sizes):
                print(f'\t\tN={N_train}')
                # Sample training covariates and response
                
                X_train = np.random.uniform(0,1,size=(N_train,3))
                X_train[:,0] = np.random.beta(alpha_pr,beta_pr,N_train)
                y_train = func.sample(X_train)
                
                true_weight = st.beta(alpha,beta).pdf(X_train[:,0])/st.beta(alpha_pr,beta_pr).pdf(X_train[:,0]) 
              
                X_tr=np.random.uniform(0,1,size=(n_tr,3))
                X_tr[:,0]=np.random.beta(alpha_pr,beta_pr,n_tr)
                X_te=np.random.uniform(0,1,size=(n_te,3))
                X_te[:,0]=np.random.beta(alpha,beta,n_te)  

                for midx, model in enumerate([m() for m in models]):
                    print(f'\t\t\t{model.label}')

    

                    model.weighted_fit(X_train, y_train, X_tr, X_te, n_tr, true_weight, 1, 3)
                    preds = model.predict(X_test)
                    print(preds.shape)

                    # Evaluate the model on the ground truth quantiles
                    l2_results[trial, scenario, midx, nidx] = ((y_quantiles - preds)**2).mean(axis=0)

                    l1_results[trial, scenario, midx, nidx] = (np.abs(y_quantiles - preds)).mean(axis=0)
                    
            print('\t', l2_results[trial, scenario])
            print('\t', l1_results[trial, scenario])